# Weather time series data set (with PyQrack quantum associative memory)

"PyQrack" is a pure Python language standard wrapper for the (C++11) Qrack quantum computer simulator library. PyQrack exposes a "quantum neuron" called "`QrackNeuron`." (Its API reference is [here](https://pyqrack.readthedocs.io/en/latest/autoapi/pyqrack/qrack_neuron/index.html).) We'd like to model a simple data set to achieve a proof-of-concept of using `QrackNeuron`.

In [1]:
#!pip install pyqrack

First, load the data set into a `pandas` dataframe.

In [2]:
import math
import random
import numpy as np
import pandas as pd

train = pd.read_csv('weather/DailyDelhiClimateTrain.csv')
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d').apply(lambda x:x.toordinal())
train['date2'] = train.loc[:,'date']

test = pd.read_csv('weather/DailyDelhiClimateTest.csv')
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d').apply(lambda x:x.toordinal())
test['date2'] = test.loc[:,'date']

print(train.head())
print("Number of observations: ", train.shape[0])

     date   meantemp   humidity  wind_speed  meanpressure   date2
0  734869  10.000000  84.500000    0.000000   1015.666667  734869
1  734870   7.400000  92.000000    2.980000   1017.800000  734870
2  734871   7.166667  87.000000    4.633333   1018.666667  734871
3  734872   8.666667  71.333333    1.233333   1017.166667  734872
4  734873   6.000000  86.833333    3.700000   1016.500000  734873
Number of observations:  1462


Separate the dependent and independent variables.

In [3]:
features = ['date', 'date2']
dependents = ['meantemp', 'humidity', 'wind_speed', 'meanpressure' ]

X = train[features]
y = train[dependents]

X_test = test[features]
y_test = test[dependents]

Ideally, we'd like to make an improvement on the goodness-of-fit of multiple linear regression, with PyQrack's `QrackNeuron`. At the very least, to show that `QrackNeuron` can be viable for modeling a data set, we'd like to show somehwat comparable performance to multiple linear regression.

PyQrack's `QrackNeuron` can only work with discrete, binary data. To model this or any data set, we have to reduce it to a simple, discrete, binary form.

We'll try to model the data set via "(quantum) associative memory." There are several statistical considerations, to avoid overfit.

Firstly, each possible discretized independent variable permutation input trains an independent parameter of a `QrackNeuron`. If a `QrackNeuron` has never seen a specific, exact permutation of input bits, it has no information about them at all, so its prediction defaults to "maximal superposition," (i.e. a totally random guess). Therefore, we'd like to keep our number of possible distinct inputs significantly fewer in number than our observation rows, when we discretize our indepedent variables.

Satisfying the first consideration, we secondly discretize our dependent variable to have exactly as many possible discrete values as possible distinct inputs. (We guess that this loses the least information about the dependent variable, while we still have enough observations to fully train our network.)

Thirdly, our learning rate should should just barely "saturate" the learned parameters of our (quantum) associative memory. As a learning volatility parameter ("`eta`") of `1/2` "fully trains" one parameter of a `QrackNeuron` between input qubits and output qubit, on average, this implies that we might set `eta` to `1/2` times `2` to the power of input qubits (summed across all predictors) divided by the number of observations. 

At a baseline, our first choice to model this data set might be multiple linear regression.

In [4]:
from sklearn import linear_model

regr = []
for i in range(len(dependents)):
    regr.append(linear_model.LinearRegression())
    regr[i].fit(X, y[dependents[i]])
    pd.DataFrame(zip(X.columns, regr[i].coef_))

y_pred = [r.predict(X_test) for r in regr]
sst = [0 for _ in regr]
ssr = [0 for _ in regr]
for i in range(len(dependents)):
    for j in range(len(y_pred)):
        sst[i] += y_test[dependents[i]][j] * y_test[dependents[i]][j]
        ssr[i] += (y_test[dependents[i]][j] - y_pred[i][j]) * (y_test[dependents[i]][j] - y_pred[i][j])
    print("Multiple linear regression validation R^2: ", 1 - ssr[i] / sst[i])

Multiple linear regression validation R^2:  0.6980203077381697
Multiple linear regression validation R^2:  0.933856389001777
Multiple linear regression validation R^2:  0.24730546944606335
Multiple linear regression validation R^2:  0.7057764776601969


To discretize the data, we split it into as many quantiles as `2` to the power of our number of input qubits. For date or time data, we'll introduce a separate parameter to control choice of quantiles, and we'll transform to the frequency domain. Fitting to frequency rather than point in time, we potentially capture periodic correlations in weather, as opposed to non-periodic changes with monotonically increasing time.

Once we have our quantiles, we bin our indepedent training and validation data.

In [5]:
in_qubit_counts = [5, 1]
out_qubit_counts = [5, 2, 1, 3]

in_tot_qubits = sum(in_qubit_counts)
in_bin_counts = [(1 << i) for i in in_qubit_counts]
in_tot_bins = sum(in_bin_counts)
in_qubits = list(range(in_tot_qubits))
out_tot_qubits = sum(out_qubit_counts)
out_bin_counts = [(1 << o) for o in out_qubit_counts]
out_tot_bins = sum(out_bin_counts)
out_qubits = list(range(in_tot_qubits, in_tot_qubits + out_tot_qubits))

In [6]:
from pyqrack import QrackSimulator, QrackNeuron

xd = []
yd = []
xd_test = []
yd_test = []
y_bins = []

for i, feature in enumerate(features):
    l = list(X[feature])
    xd_bounds_col = QrackNeuron.quantile_bounds(l, in_qubit_counts[i])
    xd.append(QrackNeuron.discretize(l, xd_bounds_col))
    xd_test.append(QrackNeuron.discretize(list(X_test[feature]), xd_bounds_col))

for i, dependent in enumerate(dependents):
    l = list(y[dependent])
    yd_bounds_col_2 = QrackNeuron.quantile_bounds(l, out_qubit_counts[i] + 1)
    yd_bounds_col = yd_bounds_col_2[0::2]
    yd.append(QrackNeuron.discretize(l, yd_bounds_col))
    yd_test.append(QrackNeuron.discretize(list(y_test[dependent]), yd_bounds_col))
    y_bins.append(yd_bounds_col_2[1::2])

xd = QrackNeuron.flatten_and_transpose(xd)
xd_test = QrackNeuron.flatten_and_transpose(xd_test)
yd = QrackNeuron.flatten_and_transpose(yd)
yd_test = QrackNeuron.flatten_and_transpose(yd_test)

We do the same for our dependent data.

We're ready to train our associative memory!

In [7]:
from IPython.display import clear_output

eta = (1 / 2) * (sum(in_bin_counts) / y.shape[0])
input_indices = list(range(in_tot_qubits))
qsim = QrackSimulator(in_tot_qubits + out_tot_qubits)

qft_qubits = list(range(in_qubit_counts[0]))

output_layer = []
for i in range(out_tot_qubits):
    output_layer.append(QrackNeuron(qsim, input_indices, in_tot_qubits + i))

# Train the network to associate powers of 2 with their log2()
print("Learning...")
for i in range(len(xd)):
    clear_output(wait=True)
    print("Epoch ", (i + 1), " out of ", len(xd))
    
    perm = xd[i]
    res = yd[i]

    for j in range(out_tot_qubits):
        qsim.reset_all()
        for k in range(in_tot_qubits):
            if perm[k]:
                qsim.x(k)
        # Transform time domain to Fourier basis
        qsim.qft(qft_qubits)
        output_layer[j].learn(eta, res[j] == 1)

Epoch  1462  out of  1462


Let's use our neural net, trained on a portion of the data, to try to predict the left-out portion of data!

In [8]:
from collections import Counter

print("Should associate each input with its trained output...")
dependents_len = len(dependents)
sum_sqr_tot = [0.0] * dependents_len
sum_sqr_res = [0.0] * dependents_len
shots = min(10 ** 6, 1 << (out_tot_qubits + 2))
for i in range(len(xd_test)):
    clear_output(wait=True)
    print("Predicting ", (i + 1), " out of ", len(xd_test))
    
    perm = xd_test[i]

    qsim.reset_all()
    for j in range(in_tot_qubits):
        if perm[j]:
            qsim.x(j)
    # Transform time domain to Fourier basis
    qsim.qft(qft_qubits)

    for j in range(out_tot_qubits):
        output_layer[j].predict()

    m_res = dict(Counter(qsim.measure_shots(out_qubits, shots)))

    front = 0
    for j in range(dependents_len):
        pred = 0
        mid_mask = out_bin_counts[j] - 1
        for k, v in m_res.items():
            pred += y_bins[j][(k >> front) & mid_mask] * v / shots
        front += out_qubit_counts[j]

        sum_sqr_tot[j] += y_test[dependents[j]][i] * y_test[dependents[j]][i]
        sum_sqr_res[j] += (y_test[dependents[j]][i] - pred) * (y_test[dependents[j]][i] - pred)

Predicting  114  out of  114


How does this compare to the validation R^2 of multiple linear regression?

In [9]:
for i in range(len(dependents)):
    print("Variable: ", dependents[i])
    print("Multiple linear regression validation R^2: ", 1 - ssr[i] / sst[i])
    print("QrackNeuron validation R^2: ", 1 - sum_sqr_res[i] / sum_sqr_tot[i])
    msr = sum_sqr_res[i] / y_test[dependents[i]].shape[0]
    print("QrackNeuron validation MSR: ", msr)
    print("QrackNeuron validation RMSE: ", math.sqrt(msr))

Variable:  meantemp
Multiple linear regression validation R^2:  0.6980203077381697
QrackNeuron validation R^2:  0.9208220287797985
QrackNeuron validation MSR:  40.503766800016564
QrackNeuron validation RMSE:  6.364256971557368
Variable:  humidity
Multiple linear regression validation R^2:  0.933856389001777
QrackNeuron validation R^2:  0.8947267205631342
QrackNeuron validation MSR:  371.13101503282184
QrackNeuron validation RMSE:  19.26476096485035
Variable:  wind_speed
Multiple linear regression validation R^2:  0.24730546944606335
QrackNeuron validation R^2:  0.6024862605278138
QrackNeuron validation MSR:  31.437241825646133
QrackNeuron validation RMSE:  5.606892350103231
Variable:  meanpressure
Multiple linear regression validation R^2:  0.7057764776601969
QrackNeuron validation R^2:  0.992039459170288
QrackNeuron validation MSR:  8088.084266439405
QrackNeuron validation RMSE:  89.93377711649502
